# Agrégation des Données Économie & Démographie pour ML

---

**Objectif** : Préparer un dataset propre avec les features économiques et démographiques pertinentes pour un modèle de prédiction des prix immobiliers.

**Niveaux d'agrégation** :
- **Département** : toutes les features disponibles
- **Commune** : revenus + population (granularité fine)

**Année de référence** : 2021 (meilleure couverture des données)

## 1. Configuration

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Chemins
PROJECT_ROOT = os.path.dirname(os.getcwd())
CLEAN_DIR = os.path.join(PROJECT_ROOT, 'data', 'clean')
OUTPUT_DIR = os.getcwd()  # Agregation_Donnees

print(f"Source : {CLEAN_DIR}")
print(f"Output : {OUTPUT_DIR}")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Inventaire des Features Pertinentes pour le ML

### Features sélectionnées pour prédire les prix immobiliers :

| Feature | Source | Granularité | Impact attendu |
|---------|--------|-------------|----------------|
| `revenu_median` | FILOSOFI 2021 | Commune | Fort (+) |
| `taux_chomage` | INSEE | Département | Moyen (-) |
| `taux_pauvrete` | FILOSOFI 2021 | Commune/Dept | Fort (-) |
| `population` | INSEE | Commune/Dept | Moyen (+) |
| `nb_menages` | FILOSOFI 2021 | Commune | Faible |
| `nb_etablissements` | Flores | Département | Moyen (+) |
| `nb_creations_entreprises` | REE | Département | Moyen (+) |
| `inflation` | IPC | National | Contexte |

In [ ]:
# Fonction de chargement
def load(name):
    return pd.read_csv(os.path.join(CLEAN_DIR, name))

# Chargement des données
print("Chargement des données...\n")

# Niveau COMMUNE (2021)
df_revenus_communes = load('revenus_communes_2021_clean.csv')
df_pauvrete_communes = load('pauvrete_communes_2021_clean.csv')

# Niveau DEPARTEMENT
df_chomage = load('chomage_departements_clean.csv')
df_emploi = load('emploi_clean.csv')
df_entreprises = load('entreprises_clean.csv')
df_population = load('population_series_clean.csv')
df_revenus_dept = load('revenus_menages_clean.csv')
df_pauvrete_dept = load('pauvrete_clean.csv')

# Niveau NATIONAL (contexte)
df_ipc = load('ipc_clean.csv')

print("Données chargées :")
print(f"  - Revenus communes : {len(df_revenus_communes):,} lignes")
print(f"  - Pauvreté communes : {len(df_pauvrete_communes):,} lignes")
print(f"  - Chômage dept : {len(df_chomage):,} lignes")
print(f"  - Emploi dept : {len(df_emploi):,} lignes")
print(f"  - Entreprises dept : {len(df_entreprises):,} lignes")

---

## 3. Agrégation Niveau COMMUNE

Dataset avec granularité communale (pour jointure avec DVF géolocalisé).

In [ ]:
# === REVENUS PAR COMMUNE ===
# Sélection des colonnes pertinentes
df_comm = df_revenus_communes[['code_commune', 'code_dept', 'nb_menages', 'nb_personnes', 'revenu_median']].copy()
df_comm.columns = ['code_commune', 'code_dept', 'nb_menages', 'population', 'revenu_median']

print(f"Revenus communes : {len(df_comm):,} communes")
df_comm.head()

In [ ]:
# === PAUVRETE PAR COMMUNE ===
df_pauv_comm = df_pauvrete_communes[['code_commune', 'taux_pauvrete']].copy()

print(f"Pauvreté communes : {len(df_pauv_comm):,} communes (beaucoup sous secret statistique)")
df_pauv_comm.head()

In [ ]:
# === FUSION NIVEAU COMMUNE ===
df_communes = df_comm.merge(df_pauv_comm, on='code_commune', how='left')

# Ajout du code département si manquant
df_communes['code_dept'] = df_communes['code_commune'].str[:2]

print(f"\nDataset COMMUNE : {len(df_communes):,} lignes")
print(f"Colonnes : {list(df_communes.columns)}")
print(f"\nValeurs manquantes :")
print(df_communes.isnull().sum())

In [ ]:
# Statistiques descriptives
print("\n=== STATISTIQUES NIVEAU COMMUNE ===")
df_communes.describe()

---

## 4. Agrégation Niveau DÉPARTEMENT

Dataset plus riche avec toutes les features disponibles.

In [ ]:
# === CHOMAGE (dernière année disponible) ===
# Filtrer 2021 ou la dernière année disponible
annee_ref = 2021

df_chom_2021 = df_chomage[df_chomage['annee'] == annee_ref].copy()
if len(df_chom_2021) == 0:
    annee_ref = df_chomage['annee'].max()
    df_chom_2021 = df_chomage[df_chomage['annee'] == annee_ref].copy()

# Moyenne annuelle par département
df_chom_dept = df_chom_2021.groupby('zone')['taux_chomage'].mean().reset_index()
df_chom_dept['code_dept'] = df_chom_dept['zone'].str.replace('D', '')
df_chom_dept = df_chom_dept[['code_dept', 'taux_chomage']]

print(f"Chômage {annee_ref} : {len(df_chom_dept)} départements")
df_chom_dept.head()

In [ ]:
# === EMPLOI (nombre d'établissements) ===
df_empl_dept = df_emploi.groupby('code_dept')['nb_etablissements'].sum().reset_index()
df_empl_dept.columns = ['code_dept', 'nb_etablissements']

print(f"Emploi : {len(df_empl_dept)} départements")
df_empl_dept.head()

In [ ]:
# === ENTREPRISES (créations) ===
df_entrep_dept = df_entreprises.groupby('code_dept')['nb_creations'].sum().reset_index()
df_entrep_dept.columns = ['code_dept', 'nb_creations_entreprises']

print(f"Entreprises : {len(df_entrep_dept)} départements")
df_entrep_dept.head()

In [ ]:
# === REVENUS PAR DEPARTEMENT (agrégé depuis communes) ===
df_rev_dept = df_revenus_communes.groupby('code_dept').agg({
    'revenu_median': 'median',
    'nb_menages': 'sum',
    'nb_personnes': 'sum'
}).reset_index()
df_rev_dept.columns = ['code_dept', 'revenu_median', 'nb_menages', 'population']

print(f"Revenus dept (agrégé) : {len(df_rev_dept)} départements")
df_rev_dept.head()

In [ ]:
# === PAUVRETE PAR DEPARTEMENT ===
# Essayer d'abord les données départementales directes
df_pauv_dept_direct = df_pauvrete_dept[df_pauvrete_dept['annee'] == df_pauvrete_dept['annee'].max()].copy()
df_pauv_dept_agg = df_pauv_dept_direct.groupby('zone')['taux_pauvrete'].mean().reset_index()
df_pauv_dept_agg['code_dept'] = df_pauv_dept_agg['zone'].str.replace('D', '')
df_pauv_dept_agg = df_pauv_dept_agg[['code_dept', 'taux_pauvrete']]

# Si peu de données, agréger depuis les communes
if len(df_pauv_dept_agg) < 50:
    df_pauv_dept_agg = df_pauvrete_communes.groupby('code_dept')['taux_pauvrete'].mean().reset_index()

print(f"Pauvreté dept : {len(df_pauv_dept_agg)} départements")
df_pauv_dept_agg.head()

In [ ]:
# === FUSION NIVEAU DEPARTEMENT ===
df_departements = df_rev_dept.copy()

# Jointures successives
df_departements = df_departements.merge(df_chom_dept, on='code_dept', how='left')
df_departements = df_departements.merge(df_pauv_dept_agg, on='code_dept', how='left')
df_departements = df_departements.merge(df_empl_dept, on='code_dept', how='left')
df_departements = df_departements.merge(df_entrep_dept, on='code_dept', how='left')

# Calcul de features dérivées
df_departements['densite_etablissements'] = (
    df_departements['nb_etablissements'] / df_departements['population'] * 1000
)
df_departements['dynamisme_entrepreneurial'] = (
    df_departements['nb_creations_entreprises'] / df_departements['population'] * 1000
)

print(f"\nDataset DEPARTEMENT : {len(df_departements)} lignes")
print(f"Colonnes : {list(df_departements.columns)}")
df_departements.head()

In [ ]:
# Valeurs manquantes
print("\n=== VALEURS MANQUANTES NIVEAU DEPARTEMENT ===")
missing = df_departements.isnull().sum()
missing_pct = (missing / len(df_departements) * 100).round(1)
pd.DataFrame({'Manquantes': missing, '%': missing_pct})

In [ ]:
# Statistiques descriptives
print("\n=== STATISTIQUES NIVEAU DEPARTEMENT ===")
df_departements.describe().round(2)

---

## 5. Enrichissement avec Features Dérivées

In [ ]:
# === NIVEAU COMMUNE : ajout des features départementales ===
# Permet d'avoir les features dept même au niveau commune

cols_dept_to_add = ['code_dept', 'taux_chomage', 'nb_etablissements', 
                    'nb_creations_entreprises', 'densite_etablissements', 
                    'dynamisme_entrepreneurial']

df_communes_enrichi = df_communes.merge(
    df_departements[cols_dept_to_add], 
    on='code_dept', 
    how='left'
)

print(f"Dataset COMMUNE enrichi : {len(df_communes_enrichi):,} lignes")
print(f"Colonnes : {list(df_communes_enrichi.columns)}")

In [ ]:
# === INFLATION NATIONALE (contexte) ===
# Dernière valeur d'inflation disponible
inflation_2021 = df_ipc[df_ipc['annee'] == 2021]['inflation_12m'].mean()
ipc_2021 = df_ipc[df_ipc['annee'] == 2021]['indice_prix'].mean()

print(f"Contexte national 2021 :")
print(f"  - Inflation moyenne : {inflation_2021:.2f}%")
print(f"  - IPC moyen : {ipc_2021:.2f}")

# Ajout au dataset département
df_departements['inflation_nationale'] = inflation_2021
df_departements['ipc_national'] = ipc_2021

# Ajout au dataset commune
df_communes_enrichi['inflation_nationale'] = inflation_2021
df_communes_enrichi['ipc_national'] = ipc_2021

---

## 6. Nettoyage Final et Export

In [ ]:
# === DATASET COMMUNE FINAL ===
# Renommage pour clarté
df_communes_final = df_communes_enrichi.copy()
df_communes_final = df_communes_final.rename(columns={
    'population': 'pop_commune'
})

# Ordre des colonnes
cols_ordre_comm = [
    'code_commune', 'code_dept',
    # Features communales
    'pop_commune', 'nb_menages', 'revenu_median', 'taux_pauvrete',
    # Features départementales
    'taux_chomage', 'nb_etablissements', 'nb_creations_entreprises',
    'densite_etablissements', 'dynamisme_entrepreneurial',
    # Contexte national
    'inflation_nationale', 'ipc_national'
]
df_communes_final = df_communes_final[[c for c in cols_ordre_comm if c in df_communes_final.columns]]

print("=== DATASET COMMUNE FINAL ===")
print(f"Dimensions : {df_communes_final.shape}")
print(f"Colonnes : {list(df_communes_final.columns)}")
df_communes_final.head()

In [ ]:
# === DATASET DEPARTEMENT FINAL ===
df_departements_final = df_departements.copy()
df_departements_final = df_departements_final.rename(columns={
    'population': 'pop_departement'
})

# Ordre des colonnes
cols_ordre_dept = [
    'code_dept',
    # Features démographiques
    'pop_departement', 'nb_menages',
    # Features économiques
    'revenu_median', 'taux_chomage', 'taux_pauvrete',
    # Features emploi/entreprises
    'nb_etablissements', 'nb_creations_entreprises',
    'densite_etablissements', 'dynamisme_entrepreneurial',
    # Contexte national
    'inflation_nationale', 'ipc_national'
]
df_departements_final = df_departements_final[[c for c in cols_ordre_dept if c in df_departements_final.columns]]

print("\n=== DATASET DEPARTEMENT FINAL ===")
print(f"Dimensions : {df_departements_final.shape}")
print(f"Colonnes : {list(df_departements_final.columns)}")
df_departements_final.head()

In [ ]:
# === EXPORT DES DATASETS ===
# Niveau commune
path_communes = os.path.join(OUTPUT_DIR, 'features_eco_demo_COMMUNE.csv')
df_communes_final.to_csv(path_communes, index=False)
print(f"Exporté : {path_communes}")
print(f"  -> {len(df_communes_final):,} communes, {df_communes_final.shape[1]} colonnes")

# Niveau département
path_dept = os.path.join(OUTPUT_DIR, 'features_eco_demo_DEPARTEMENT.csv')
df_departements_final.to_csv(path_dept, index=False)
print(f"\nExporté : {path_dept}")
print(f"  -> {len(df_departements_final)} départements, {df_departements_final.shape[1]} colonnes")

---

## 7. Documentation pour les Collègues

In [ ]:
# === DICTIONNAIRE DES VARIABLES ===
dictionnaire = """
============================================================
DICTIONNAIRE DES VARIABLES - ÉCONOMIE & DÉMOGRAPHIE
============================================================

FICHIER : features_eco_demo_COMMUNE.csv
CLÉ DE JOINTURE : code_commune (5 caractères, ex: 01001, 75056)

FICHIER : features_eco_demo_DEPARTEMENT.csv  
CLÉ DE JOINTURE : code_dept (2-3 caractères, ex: 01, 75, 2A)

------------------------------------------------------------
VARIABLES DISPONIBLES :
------------------------------------------------------------

IDENTIFIANTS :
  - code_commune    : Code INSEE commune (5 car.)
  - code_dept       : Code département (2-3 car.)

DÉMOGRAPHIE :
  - pop_commune     : Population de la commune (nb personnes)
  - pop_departement : Population du département
  - nb_menages      : Nombre de ménages fiscaux

ÉCONOMIE - REVENUS :
  - revenu_median   : Revenu disponible médian (€/an)
  - taux_pauvrete   : Taux de pauvreté à 60% du revenu médian (%)

ÉCONOMIE - EMPLOI :
  - taux_chomage    : Taux de chômage départemental (%)
  - nb_etablissements : Nombre d'établissements (dept)
  - nb_creations_entreprises : Créations d'entreprises (dept)

INDICATEURS DÉRIVÉS :
  - densite_etablissements : Établissements pour 1000 hab.
  - dynamisme_entrepreneurial : Créations pour 1000 hab.

CONTEXTE NATIONAL :
  - inflation_nationale : Taux d'inflation annuel (%)
  - ipc_national : Indice des prix à la consommation

------------------------------------------------------------
ANNÉE DE RÉFÉRENCE : 2021
SOURCE : INSEE (API pynsee) + FILOSOFI 2021
------------------------------------------------------------
"""
print(dictionnaire)

# Sauvegarder le dictionnaire
with open(os.path.join(OUTPUT_DIR, 'DICTIONNAIRE_VARIABLES.txt'), 'w') as f:
    f.write(dictionnaire)

In [ ]:
# === RÉSUMÉ STATISTIQUE POUR LES COLLÈGUES ===
print("\n" + "="*60)
print("RÉSUMÉ - DONNÉES ÉCONOMIE & DÉMOGRAPHIE POUR ML")
print("="*60)

print("\n📁 FICHIERS GÉNÉRÉS :")
print(f"   1. features_eco_demo_COMMUNE.csv")
print(f"      -> {len(df_communes_final):,} communes")
print(f"      -> Clé : code_commune")
print(f"   2. features_eco_demo_DEPARTEMENT.csv")
print(f"      -> {len(df_departements_final)} départements")
print(f"      -> Clé : code_dept")

print("\n📊 FEATURES POUR LE ML :")
print("   Variables numériques prêtes à utiliser :")
for col in df_departements_final.select_dtypes(include=[np.number]).columns:
    print(f"     - {col}")

print("\n🔗 JOINTURES POSSIBLES :")
print("   - Avec DVF : sur code_commune (5 car.)")
print("   - Avec Sécurité : sur code_commune (CODGEO_2025)")
print("   - Avec Éducation : sur code_commune ou GPS")
print("   - Conversion : code_dept = code_commune[:2]")

print("\n⚠️  VALEURS MANQUANTES :")
print(f"   - taux_pauvrete : {df_communes_final['taux_pauvrete'].isnull().sum():,} communes (secret stat.)")
print(f"   - Autres features : complètes")

print("\n" + "="*60)

---

## 8. Aperçu Final des Données

In [ ]:
# Aperçu niveau commune
print("=== ÉCHANTILLON NIVEAU COMMUNE ===")
display(df_communes_final.sample(10))

In [ ]:
# Aperçu niveau département
print("=== ÉCHANTILLON NIVEAU DÉPARTEMENT ===")
display(df_departements_final.sample(10))

In [ ]:
# Corrélations entre features (niveau dept)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
cols_num = df_departements_final.select_dtypes(include=[np.number]).columns
corr = df_departements_final[cols_num].corr()

mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdYlGn', center=0)
plt.title('Corrélations entre Features Économiques & Démographiques')
plt.tight_layout()
plt.show()

---

## Conclusion

**Deux datasets prêts pour le ML** :

1. **`features_eco_demo_COMMUNE.csv`** (~31k lignes)
   - Granularité fine pour jointure avec DVF géolocalisé
   - Clé : `code_commune`

2. **`features_eco_demo_DEPARTEMENT.csv`** (~100 lignes)
   - Toutes les features agrégées
   - Clé : `code_dept`

**Prochaines étapes** :
- Fusionner avec les données Sécurité (collègue)
- Fusionner avec les données Éducation (collègue)
- Fusionner avec DVF (prix immobiliers) → variable cible

---

*Notebook créé pour le projet Compagnon Immobilier - DataScientest*